In [1]:
# import warnings
# warnings.filterwarnings("ignore")

import csv
import numpy as np
import pandas as pd
import scipy

import os, os.path
import sys

# to move files from one directory to another
import shutil

In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
import NASA_core as nc

In [3]:
data_dir = "/Users/hn/Documents/01_research_data/NASA/data_part_of_shapefile/"
output_dir = "/Users/hn/Documents/01_research_data/NASA/"
param_dir = output_dir + "/parameters/"

# double_crop_potens = pd.read_csv(param_dir + "double_crop_potential_plants.csv")

In [4]:
f_names = ["AdamBenton2016.csv",
           "FranklinYakima2018.csv",
           "Grant2017.csv",
           "Walla2015.csv"]

# List of unwanted fields

In [5]:
unwanted_crop = ["christmas tree", "conifer seed", 
                 "crp/conservation", "dahlia", "dandelion", "developed", 
                 "driving range", "flowers, nursery and christmas tree farms",
                 "golf course", "greenhouse", "iris", "miscellaneous deciduous", 
                 "nursery, caneberry", "nursery, greenhouse", "nursery, holly",
                 "nursery, lavender", "nursery, lilac", "nursery, orchard/vineyard",
                 "nursery, ornamental", "nursery, silvaculture", "nursery, silviculture",
                 "peony", "reclamation seed",
                 "research station", "shellfish", 
                 "silvaculture", "silviculture", "tulip"]

# Read Files

In [6]:
all_fields = pd.DataFrame(data=None, index=None)
all_fields_correct_year = pd.DataFrame(data=None, index=None)
for file in f_names:
    curr_table = pd.read_csv(data_dir + file, low_memory=False)
    all_fields = pd.concat([all_fields, curr_table]).reset_index(drop=True)
    
    # pick proper year here. currently it is in the function 
    # generate_training_set_important_counties(.)
    # or you want to keep it in the function?
    curr_table = nc.filter_by_lastSurvey(curr_table, file[-8:-4])
    all_fields_correct_year = pd.concat([all_fields_correct_year, curr_table]).reset_index(drop=True)
    

all_fields['CropTyp'] = all_fields['CropTyp'].str.lower()
all_fields.drop_duplicates(inplace=True) # sanity check


all_fields_correct_year['CropTyp'] = all_fields_correct_year['CropTyp'].str.lower()
all_fields_correct_year.drop_duplicates(inplace=True) # sanity check

In [7]:
print ("all_fields.shape is: ", all_fields.shape) 
print ("all_fields_correct_year.shape is: ", all_fields_correct_year.shape) 

print ("No. unique crops in all_fields is:", len(all_fields.CropTyp.unique()))
print ("No. unique crops in all_fields_correct_year is:", len(all_fields.CropTyp.unique()))

all_fields.shape is:  (77530, 15)
all_fields_correct_year.shape is:  (64846, 15)
No. unique crops in all_fields is: 116
No. unique crops in all_fields_correct_year is: 116


# Filters 
  - NASS, last survey date, Irrigated fields, and unwanted locations such as research station.

# Filter Irrigated Fields

In [8]:
# pick up irrigated
all_fields_irr = nc.filter_out_nonIrrigated(all_fields)
all_fields_correct_year_irr = nc.filter_out_nonIrrigated(all_fields_correct_year)


print ("all_fields.shape:", all_fields.shape)
print ("all_fields_irr.shape:", all_fields_irr.shape)
print ("")
print ("all_fields_correct_year.shape:", all_fields_correct_year.shape)
print ("all_fields_correct_year_irr.shape", all_fields_correct_year_irr.shape)

all_fields.shape: (77530, 15)
all_fields_irr.shape: (53582, 15)

all_fields_correct_year.shape: (64846, 15)
all_fields_correct_year_irr.shape (44850, 15)


In [9]:
del(all_fields, all_fields_correct_year)

In [10]:
all_fields_correct_year_irr.shape

(44850, 15)

In [11]:
all_fields_correct_year_irr.head(2)

,ID,CropGrp,CropTyp,Acres,Irrigtn,IntlSrD,LstSrvD,DataSrc,Notes,TRS,county,RtCrpTy,ExctAcr,Shp_Lng,Shap_Ar
6,51380_WSDA_SF_2016,Seed,bluegrass seed,6,sprinkler,2003/07/01 00:00:00,2016/09/19 00:00:00,wsda,NaN,T20R35E24,Adams,Wheat,6.116621,937.384083,24753.087193
7,51381_WSDA_SF_2016,Cereal Grain,wheat,117,center pivot,2003/07/01 00:00:00,2016/09/20 00:00:00,wsda,NaN,T16R36E22,Adams,Potato,116.743101,2445.413312,472442.568255


# Toss NASS

In [12]:
print (all_fields_correct_year_irr.shape)
all_fields_correct_year_irr_noNass = nc.filter_out_NASS(all_fields_correct_year_irr)
print (all_fields_correct_year_irr_noNass.shape)

(44850, 15)
(37210, 15)


# Drop stupid Columns

In [15]:
all_fields_correct_year_irr_noNass.drop(labels=["CropGrp", "CropTyp", "IntlSrD", "Notes", "TRS",
                                                "RtCrpTy", "Shp_Lng", "Shap_Ar", "ExctAcr"], 
                                        axis='columns',
                                       inplace = True)

In [22]:
all_correct_year_irr_noNass_count = all_fields_correct_year_irr_noNass.groupby(['county', 'Irrigtn']).count()
all_correct_year_irr_noNass_count.reset_index(inplace=True)
all_correct_year_irr_noNass_count.sort_values(by=['county', 'Irrigtn'], inplace=True)

In [28]:
all_correct_year_irr_noNass_count.head(5)

,county,Irrigtn,ID,Acres,LstSrvD,DataSrc,ExctAcr
0,Adams,big gun,6,6,6,6,6
1,Adams,big gun/center pivot,2,2,2,2,2
2,Adams,big gun/sprinkler,1,1,1,1,1
3,Adams,center pivot,1852,1852,1852,1852,1852
4,Adams,center pivot/drip,4,4,4,4,4


In [34]:
all_correct_year_irr_noNass_count.drop(labels=["Acres", "LstSrvD", "DataSrc"], 
                                       axis='columns',
                                       inplace = True)
all_correct_year_irr_noNass_count.rename(columns={"ID": "count"}, inplace=True)

all_correct_year_irr_noNass_count.head(2)

,county,Irrigtn,count
0,Adams,big gun,6
1,Adams,big gun/center pivot,2


In [24]:
all_correct_year_irr_noNass_Acr = all_fields_correct_year_irr_noNass.groupby(['county', 'Irrigtn']).sum()
all_correct_year_irr_noNass_Acr.reset_index(inplace=True)
all_correct_year_irr_noNass_Acr.sort_values(by=['county', 'Irrigtn'], inplace=True)

In [36]:
all_correct_year_irr_noNass_Acr.head(2)

,county,Irrigtn,Acres
0,Adams,big gun,177
1,Adams,big gun/center pivot,5


In [37]:
print (all_correct_year_irr_noNass_Acr.shape)
print (all_correct_year_irr_noNass_count.shape)

(114, 3)

In [39]:
stats = pd.merge(all_correct_year_irr_noNass_count, all_correct_year_irr_noNass_Acr, 
                 on=['county', "Irrigtn"], how='left')
stats.head(2)

In [41]:
out_name = output_dir + "rill_wheel_stat_allFields_CountyIrrType.csv"
stats.to_csv(out_name, index = False)
del(stats)

# Group by irrigation type. No counties here

In [44]:
all_correct_year_irr_noNass_count = all_fields_correct_year_irr_noNass.groupby(['Irrigtn']).count()
all_correct_year_irr_noNass_count.reset_index(inplace=True)
all_correct_year_irr_noNass_count.sort_values(by=['Irrigtn'], inplace=True)

all_correct_year_irr_noNass_count.drop(labels=["Acres", "LstSrvD", "DataSrc", "county", "ExctAcr"], 
                                       axis='columns',
                                       inplace = True)
all_correct_year_irr_noNass_count.rename(columns={"ID": "count"}, inplace=True)

all_correct_year_irr_noNass_count.head(2)

,Irrigtn,count
0,big gun,125
1,big gun/center pivot,6


In [48]:
all_correct_year_irr_noNass_Acr = all_fields_correct_year_irr_noNass.groupby(['Irrigtn']).sum()
all_correct_year_irr_noNass_Acr.reset_index(inplace=True)
all_correct_year_irr_noNass_Acr.sort_values(by=['Irrigtn'], inplace=True)

all_correct_year_irr_noNass_Acr.head(2)

,Irrigtn,Acres
0,big gun,1702
1,big gun/center pivot,126


In [49]:
stats = pd.merge(all_correct_year_irr_noNass_count, all_correct_year_irr_noNass_Acr, 
                 on=["Irrigtn"], how='left')
stats.head(2)

,Irrigtn,count,Acres
0,big gun,125,1702
1,big gun/center pivot,6,126


In [50]:
out_name = output_dir + "rill_wheel_stat_allFields_justIrrType.csv"
stats.to_csv(out_name, index = False)

In [ ]:
del(all_correct_year_irr_noNass_Acr, 
    all_correct_year_irr_noNass_count, 
    all_fields_correct_year_irr,
    all_fields_irr, all_fields_correct_year_irr_noNass, stats)

# Do the same for the damn 6000 training set

In [78]:
adir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
fucking_train = pd.read_csv(adir + "evaluation_set.csv")

In [79]:
fucking_train.drop(labels=['ID', "ExctAcr", "LstSrvD", "DataSrc", "CropTyp"], 
                   axis="columns", 
                   inplace=True)
fucking_train.head(2)

,Irrigtn,Acres,county
0,center pivot,34,Grant
1,center pivot,62,Grant


In [80]:
fucking_train_count = fucking_train.groupby(['county', 'Irrigtn']).count()
fucking_train_count.reset_index(inplace=True)
fucking_train_count.sort_values(by=['county', 'Irrigtn'], inplace=True)
fucking_train_count.rename(columns={"Acres": "count"}, inplace=True)
fucking_train_count.head(2)

,county,Irrigtn,count
0,Adams,big gun,4
1,Adams,center pivot,332


In [81]:
fucking_train_Acr = fucking_train.groupby(['county', 'Irrigtn']).sum()
fucking_train_Acr.reset_index(inplace=True)
fucking_train_Acr.sort_values(by=['county', 'Irrigtn'], inplace=True)
fucking_train_Acr.head(2)

,county,Irrigtn,Acres
0,Adams,big gun,70
1,Adams,center pivot,24223


In [82]:
stats = pd.merge(fucking_train_count, fucking_train_Acr, on=['county', "Irrigtn"], how='left')
stats.head(2)

,county,Irrigtn,count,Acres
0,Adams,big gun,4,70
1,Adams,center pivot,332,24223


In [83]:
out_name = output_dir + "rill_wheel_stat_trainSet_CountyIrrType.csv"
stats.to_csv(out_name, index = False)
del(stats)

# No county here

In [84]:
fucking_train_count = fucking_train.groupby(['Irrigtn']).count()
fucking_train_count.reset_index(inplace=True)
fucking_train_count.sort_values(by=['Irrigtn'], inplace=True)
fucking_train_count.drop(labels=["county"], axis="columns", inplace=True)

fucking_train_count.rename(columns={"Acres": "count"}, inplace=True)
fucking_train_count.head(2)

,Irrigtn,count
0,big gun,32
1,big gun/sprinkler,4


In [85]:
fucking_train_Acr = fucking_train.groupby(['Irrigtn']).sum()
fucking_train_Acr.reset_index(inplace=True)
fucking_train_Acr.sort_values(by=['Irrigtn'], inplace=True)
fucking_train_Acr.head(2)

,Irrigtn,Acres
0,big gun,664
1,big gun/sprinkler,79


In [86]:
stats = pd.merge(fucking_train_count, fucking_train_Acr, on=["Irrigtn"], how='left')
stats.head(2)

,Irrigtn,count,Acres
0,big gun,32,664
1,big gun/sprinkler,4,79


In [87]:
out_name = output_dir + "rill_wheel_stat_trainSet_JustIrrType.csv"
stats.to_csv(out_name, index = False)

In [104]:
del(fucking_train_count, fucking_train_Acr, stats)

# Do the same fucking shit for 250 experts' fields

In [109]:
import pandas as pd
import csv

import os, os.path
import sys

# to move files from one directory to another
import shutil

In [110]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"

choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choices_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names
print (choices_set_1_sheet_names)


['extended_1', 'extended_2', 'extended_3', 'extended_4', 'extended_5', 'extended_6']


In [111]:
experts_250 = pd.DataFrame()

for a_sheet in choices_set_1_sheet_names:
    a_choice_sheet = choices_set_1_xl.parse(a_sheet)
    experts_250 = pd.concat([experts_250, a_choice_sheet]).reset_index(drop=True)

In [112]:
experts_250.drop(labels = ["Question_in_set", "Question_overall", "ID", 
                           "NDVI_TS_Name", "corrected_RGB", "TOA_RGB", "latitude",
                           "longitude", "CropTyp", "DataSrc", "ExctAcr", "LstSrvD"],
                 axis="columns",
                inplace=True)
experts_250.head(2)

,Irrigtn,Acres,county
0,center pivot,117,Benton
1,center pivot,71,Benton


In [114]:
experts_250_count = experts_250.groupby(['county', 'Irrigtn']).count()
experts_250_count.reset_index(inplace=True)
experts_250_count.sort_values(by=['county', 'Irrigtn'], inplace=True)
experts_250_count.rename(columns={"Acres": "count"}, inplace=True)
print (experts_250_count.head(2))

experts_250_Acr = experts_250.groupby(['county', 'Irrigtn']).sum()
experts_250_Acr.reset_index(inplace=True)
experts_250_Acr.sort_values(by=['county', 'Irrigtn'], inplace=True)
experts_250_Acr.head(2)

  county       Irrigtn  count
0  Adams  center pivot     46
1  Adams          rill     10


,county,Irrigtn,Acres
0,Adams,center pivot,3965
1,Adams,rill,182


In [117]:
stats = pd.merge(experts_250_Acr, experts_250_count, on=["county", "Irrigtn"], how='left')

out_name = output_dir + "rill_wheel_stat_expertsSet1_CountyIrrType.csv"
stats.to_csv(out_name, index = False)
del(stats)

In [125]:
experts_250_count = experts_250.groupby(['Irrigtn']).count()
experts_250_count.reset_index(inplace=True)
experts_250_count.drop(labels=["county"], axis="columns", inplace=True)
experts_250_count.sort_values(by=['Irrigtn'], inplace=True)
experts_250_count.rename(columns={"Acres": "count"}, inplace=True)
print (experts_250_count.head(2))

experts_250_Acr = experts_250.groupby(['Irrigtn']).sum()
experts_250_Acr.reset_index(inplace=True)
experts_250_Acr.sort_values(by=['Irrigtn'], inplace=True)
experts_250_Acr.head(2)



                  Irrigtn  count
0            center pivot    200
1  center pivot/sprinkler      3


,Irrigtn,Acres
0,center pivot,13959
1,center pivot/sprinkler,315


In [127]:
stats = pd.merge(experts_250_Acr, experts_250_count, on=["Irrigtn"], how='left')

out_name = output_dir + "rill_wheel_stat_expertsSet1_JustIrrType.csv"
stats.to_csv(out_name, index = False)
del(stats)